In [5]:
import requests

In [6]:
import requests

url = 'https://query.wikidata.org/sparql'
query = '''
SELECT  ?law ?human1 ?human1Label ?human2 ?human2Label ?nat1 ?nat2 ?o1 ?o2 ?lawLabel ?type
WHERE {
  ?law wdt:P31 ?type .
  
  ?human1 wdt:P31 wd:Q5 .
  ?human2 wdt:P31 wd:Q5 .
  FILTER(?human1 != ?human2) .
  ?law wdt:P138 ?human1, ?human2.
  ?human1 wdt:P21 wd:Q6581072.
  ?human2 wdt:P21 wd:Q6581097.
  
  ?human2 wdt:P27 ?nat2.
  ?human1 wdt:P27 ?nat1.
  
  ?law ^schema:about ?article .
    ?article schema:isPartOf <https://en.wikipedia.org/>; schema:name ?articlename .
  ?human2 wdt:P106 ?o2.
  ?human1 wdt:P106 ?o1.
  
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
  }
} LIMIT 50000
'''
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()
output_filename = "wikidata_things_named_after_two_mixed_gender.json"

with open(output_filename,"w") as file:
    json.dump(data,file)

In [7]:
import json
input_filename = "wikidata_things_named_after_two_mixed_gender.json"
with open(input_filename,"r") as file:
    data = json.load(file)

In [8]:
cleaned_data = data['results']['bindings']
for binding in cleaned_data:
    for k,v in binding.items():
        binding[k] = v['value']

In [13]:
import pandas as pd 
wikidata_de_ents_dataframe = pd.DataFrame(cleaned_data)
wikidata_de_ents_dataframe = wikidata_de_ents_dataframe.groupby("lawLabel").agg(lambda x: x if len(x) == 1 else set(x))

In [14]:
wikidata_de_ents_dataframe

,law,human1,human2,nat2,nat1,o2,o1,type,human1Label,human2Label
lawLabel,,,,,,,,,,
135268 Haigneré,{http://www.wikidata.org/entity/Q2641086},{http://www.wikidata.org/entity/Q235580},{http://www.wikidata.org/entity/Q721397},{http://www.wikidata.org/entity/Q142},{http://www.wikidata.org/entity/Q142},"{http://www.wikidata.org/entity/Q11631, http:/...","{http://www.wikidata.org/entity/Q11631, http:/...",{http://www.wikidata.org/entity/Q3863},{Claudie Haigneré},{Jean-Pierre Haigneré}
2325 Chernykh,{http://www.wikidata.org/entity/Q516852},{http://www.wikidata.org/entity/Q234878},{http://www.wikidata.org/entity/Q318611},"{http://www.wikidata.org/entity/Q212, http://w...","{http://www.wikidata.org/entity/Q212, http://w...",{http://www.wikidata.org/entity/Q11063},{http://www.wikidata.org/entity/Q11063},{http://www.wikidata.org/entity/Q3863},{Lyudmila Chernykh},{Nikolai Chernykh}
3195 Fedchenko,{http://www.wikidata.org/entity/Q666223},{http://www.wikidata.org/entity/Q3350415},"{http://www.wikidata.org/entity/Q258817, http:...",{http://www.wikidata.org/entity/Q34266},{http://www.wikidata.org/entity/Q34266},"{http://www.wikidata.org/entity/Q2374149, http...","{http://www.wikidata.org/entity/Q483501, http:...",{http://www.wikidata.org/entity/Q3863},{Olga Fedchenko},"{Alexei Fedchenko, Boris Fedtschenko}"
3484 Neugebauer,{http://www.wikidata.org/entity/Q657366},{http://www.wikidata.org/entity/Q4316384},"{http://www.wikidata.org/entity/Q111996, http:...","{http://www.wikidata.org/entity/Q30, http://ww...",{http://www.wikidata.org/entity/Q30},"{http://www.wikidata.org/entity/Q16063546, htt...","{http://www.wikidata.org/entity/Q12094958, htt...",{http://www.wikidata.org/entity/Q3863},{Marcia Neugebauer},"{Gerald Neugebauer, Otto E. Neugebauer}"
4242 Brecher,{http://www.wikidata.org/entity/Q153142},{http://www.wikidata.org/entity/Q108482630},{http://www.wikidata.org/entity/Q3195167},{http://www.wikidata.org/entity/Q30},"{http://www.wikidata.org/entity/Q30, http://ww...",{http://www.wikidata.org/entity/Q11063},{http://www.wikidata.org/entity/Q109123470},{http://www.wikidata.org/entity/Q3863},{Aviva Brecher},{Kenneth Brecher}
...,...,...,...,...,...,...,...,...,...,...
Wolf Prize,{http://www.wikidata.org/entity/Q739936},{http://www.wikidata.org/entity/Q16009735},{http://www.wikidata.org/entity/Q215899},"{http://www.wikidata.org/entity/Q241, http://w...",{http://www.wikidata.org/entity/Q29},"{http://www.wikidata.org/entity/Q193391, http:...",{http://www.wikidata.org/entity/Q10833314},{http://www.wikidata.org/entity/Q11448906},{Francisca Subirana},{Ricardo Wolf}
Zsa Zsa Speck,{http://www.wikidata.org/entity/Q2734962},{http://www.wikidata.org/entity/Q207405},{http://www.wikidata.org/entity/Q944350},{http://www.wikidata.org/entity/Q30},"{http://www.wikidata.org/entity/Q600018, http:...","{http://www.wikidata.org/entity/Q2159907, http...","{http://www.wikidata.org/entity/Q2259451, http...",{http://www.wikidata.org/entity/Q5},{Zsa Zsa Gabor},{Richard Speck}
curie,{http://www.wikidata.org/entity/Q229354},{http://www.wikidata.org/entity/Q7186},{http://www.wikidata.org/entity/Q37463},{http://www.wikidata.org/entity/Q142},"{http://www.wikidata.org/entity/Q142, http://w...","{http://www.wikidata.org/entity/Q1622272, http...","{http://www.wikidata.org/entity/Q1622272, http...","{http://www.wikidata.org/entity/Q47574, http:/...",{Marie Curie},{Pierre Curie}


In [15]:
def get_extract(wiki_title,debug=False):
    url='https://en.wikipedia.org/w/api.php'
    
    params  = {
        'format': 'json', 'action': 'query','prop':'extracts&exintro&explaintext','exintro':None,'explaintext':None,
        'titles':wiki_title,'redirects':1}
    params = '&'.join([k if v is None else f"{k}={v}" for k, v in params.items()])

    r = requests.get(url, params=params)
    data = r.json()
    if debug:
        print(r.url)
#     print(data['query']['pages'])
    for k,v in data['query']['pages'].items():
        return v['extract']
#     return data['query']['pages']['title']

In [62]:
get_extract('B²FH paper',True)

https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&exintro&explaintext&titles=B%C2%B2FH%20paper&redirects=1


KeyError: 'extract'

In [75]:
wikidata_de_ents_dataframe.loc['B²FH paper','lawLabel'] = 'B2FH paper'

In [16]:

failed_indices = []
for index, row in wikidata_de_ents_dataframe.iterrows():
    try:
        wikidata_de_ents_dataframe.loc[index,'extract']  = get_extract(index)
    except Exception as e :
        failed_indices.append((index,e))
        

In [17]:
wikidata_de_ents_dataframe

,law,human1,human2,nat2,nat1,o2,o1,type,human1Label,human2Label,extract
lawLabel,,,,,,,,,,,
135268 Haigneré,{http://www.wikidata.org/entity/Q2641086},{http://www.wikidata.org/entity/Q235580},{http://www.wikidata.org/entity/Q721397},{http://www.wikidata.org/entity/Q142},{http://www.wikidata.org/entity/Q142},"{http://www.wikidata.org/entity/Q11631, http:/...","{http://www.wikidata.org/entity/Q11631, http:/...",{http://www.wikidata.org/entity/Q3863},{Claudie Haigneré},{Jean-Pierre Haigneré},The following is a partial list of minor plane...
2325 Chernykh,{http://www.wikidata.org/entity/Q516852},{http://www.wikidata.org/entity/Q234878},{http://www.wikidata.org/entity/Q318611},"{http://www.wikidata.org/entity/Q212, http://w...","{http://www.wikidata.org/entity/Q212, http://w...",{http://www.wikidata.org/entity/Q11063},{http://www.wikidata.org/entity/Q11063},{http://www.wikidata.org/entity/Q3863},{Lyudmila Chernykh},{Nikolai Chernykh},"2325 Chernykh, provisional designation 1979 SP..."
3195 Fedchenko,{http://www.wikidata.org/entity/Q666223},{http://www.wikidata.org/entity/Q3350415},"{http://www.wikidata.org/entity/Q258817, http:...",{http://www.wikidata.org/entity/Q34266},{http://www.wikidata.org/entity/Q34266},"{http://www.wikidata.org/entity/Q2374149, http...","{http://www.wikidata.org/entity/Q483501, http:...",{http://www.wikidata.org/entity/Q3863},{Olga Fedchenko},"{Alexei Fedchenko, Boris Fedtschenko}",The following is a partial list of minor plane...
3484 Neugebauer,{http://www.wikidata.org/entity/Q657366},{http://www.wikidata.org/entity/Q4316384},"{http://www.wikidata.org/entity/Q111996, http:...","{http://www.wikidata.org/entity/Q30, http://ww...",{http://www.wikidata.org/entity/Q30},"{http://www.wikidata.org/entity/Q16063546, htt...","{http://www.wikidata.org/entity/Q12094958, htt...",{http://www.wikidata.org/entity/Q3863},{Marcia Neugebauer},"{Gerald Neugebauer, Otto E. Neugebauer}",The following is a partial list of minor plane...
4242 Brecher,{http://www.wikidata.org/entity/Q153142},{http://www.wikidata.org/entity/Q108482630},{http://www.wikidata.org/entity/Q3195167},{http://www.wikidata.org/entity/Q30},"{http://www.wikidata.org/entity/Q30, http://ww...",{http://www.wikidata.org/entity/Q11063},{http://www.wikidata.org/entity/Q109123470},{http://www.wikidata.org/entity/Q3863},{Aviva Brecher},{Kenneth Brecher},The following is a partial list of minor plane...
...,...,...,...,...,...,...,...,...,...,...,...
Wolf Prize,{http://www.wikidata.org/entity/Q739936},{http://www.wikidata.org/entity/Q16009735},{http://www.wikidata.org/entity/Q215899},"{http://www.wikidata.org/entity/Q241, http://w...",{http://www.wikidata.org/entity/Q29},"{http://www.wikidata.org/entity/Q193391, http:...",{http://www.wikidata.org/entity/Q10833314},{http://www.wikidata.org/entity/Q11448906},{Francisca Subirana},{Ricardo Wolf},The Wolf Prize is an international award grant...
Zsa Zsa Speck,{http://www.wikidata.org/entity/Q2734962},{http://www.wikidata.org/entity/Q207405},{http://www.wikidata.org/entity/Q944350},{http://www.wikidata.org/entity/Q30},"{http://www.wikidata.org/entity/Q600018, http:...","{http://www.wikidata.org/entity/Q2159907, http...","{http://www.wikidata.org/entity/Q2259451, http...",{http://www.wikidata.org/entity/Q5},{Zsa Zsa Gabor},{Richard Speck},Marilyn Manson is an American rock band formed...
curie,{http://www.wikidata.org/entity/Q229354},{http://www.wikidata.org/entity/Q7186},{http://www.wikidata.org/entity/Q37463},{http://www.wikidata.org/entity/Q142},"{http://www.wikidata.org/entity/Q142, http://w...","{http://www.wikidata.org/entity/Q1622272, http...","{http://www.wikidata.org/entity/Q1622272, http...","{http://www.wikidata.org/entity/Q47574, http:/...",{Marie Curie},{Pierre Curie},"Curie may refer to:\n\nCurie family, a family ..."


In [5]:
pd.from_csv("temp_mixed.csv")

AttributeError: module 'pandas' has no attribute 'from_csv'

In [85]:
# wikidata_de_ents_dataframe
wikidata_de_ents_dataframe[wikidata_de_ents_dataframe['extract'].notna()]


,law,human1,human2,nat2,nat1,o2,o1,type,lawLabel,extract
lawLabel,,,,,,,,,,
135268 Haigneré,{http://www.wikidata.org/entity/Q2641086},{http://www.wikidata.org/entity/Q235580},{http://www.wikidata.org/entity/Q721397},{http://www.wikidata.org/entity/Q142},{http://www.wikidata.org/entity/Q142},"{http://www.wikidata.org/entity/Q11631, http:/...","{http://www.wikidata.org/entity/Q39631, http:/...",{http://www.wikidata.org/entity/Q3863},NaN,The following is a partial list of minor plane...
2325 Chernykh,{http://www.wikidata.org/entity/Q516852},{http://www.wikidata.org/entity/Q234878},{http://www.wikidata.org/entity/Q318611},"{http://www.wikidata.org/entity/Q212, http://w...","{http://www.wikidata.org/entity/Q15180, http:/...",{http://www.wikidata.org/entity/Q11063},{http://www.wikidata.org/entity/Q11063},{http://www.wikidata.org/entity/Q3863},NaN,"2325 Chernykh, provisional designation 1979 SP..."
3195 Fedchenko,{http://www.wikidata.org/entity/Q666223},{http://www.wikidata.org/entity/Q3350415},"{http://www.wikidata.org/entity/Q2910947, http...",{http://www.wikidata.org/entity/Q34266},{http://www.wikidata.org/entity/Q34266},"{http://www.wikidata.org/entity/Q3055126, http...","{http://www.wikidata.org/entity/Q483501, http:...",{http://www.wikidata.org/entity/Q3863},NaN,The following is a partial list of minor plane...
3484 Neugebauer,{http://www.wikidata.org/entity/Q657366},{http://www.wikidata.org/entity/Q4316384},"{http://www.wikidata.org/entity/Q214345, http:...","{http://www.wikidata.org/entity/Q40, http://ww...",{http://www.wikidata.org/entity/Q30},"{http://www.wikidata.org/entity/Q169470, http:...","{http://www.wikidata.org/entity/Q12094958, htt...",{http://www.wikidata.org/entity/Q3863},NaN,The following is a partial list of minor plane...
4242 Brecher,{http://www.wikidata.org/entity/Q153142},{http://www.wikidata.org/entity/Q108482630},{http://www.wikidata.org/entity/Q3195167},{http://www.wikidata.org/entity/Q30},"{http://www.wikidata.org/entity/Q218, http://w...",{http://www.wikidata.org/entity/Q11063},{http://www.wikidata.org/entity/Q109123470},{http://www.wikidata.org/entity/Q3863},NaN,The following is a partial list of minor plane...
...,...,...,...,...,...,...,...,...,...,...
Winfrenatia,{http://www.wikidata.org/entity/Q8025051},{http://www.wikidata.org/entity/Q36668722},{http://www.wikidata.org/entity/Q19965708},{http://www.wikidata.org/entity/Q183},{http://www.wikidata.org/entity/Q183},"{http://www.wikidata.org/entity/Q1622272, http...",{http://www.wikidata.org/entity/Q30059961},{http://www.wikidata.org/entity/Q16521},NaN,Winfrenatia is the oldest known terrestrial li...
Wolf Prize,{http://www.wikidata.org/entity/Q739936},{http://www.wikidata.org/entity/Q16009735},{http://www.wikidata.org/entity/Q215899},"{http://www.wikidata.org/entity/Q43287, http:/...",{http://www.wikidata.org/entity/Q29},"{http://www.wikidata.org/entity/Q593644, http:...",{http://www.wikidata.org/entity/Q10833314},{http://www.wikidata.org/entity/Q11448906},NaN,The Wolf Prize is an international award grant...
Zsa Zsa Speck,{http://www.wikidata.org/entity/Q2734962},{http://www.wikidata.org/entity/Q207405},{http://www.wikidata.org/entity/Q944350},{http://www.wikidata.org/entity/Q30},"{http://www.wikidata.org/entity/Q600018, http:...","{http://www.wikidata.org/entity/Q2159907, http...","{http://www.wikidata.org/entity/Q2259451, http...",{http://www.wikidata.org/entity/Q5},NaN,Marilyn Manson is an American rock band formed...


In [86]:
wikidata_de_ents_dataframe.to_csv("temp_mixed.csv")

In [50]:
extracts[1]

'Clintonism refers to the political and economic policies of Bill Clinton and Hillary Rodham Clinton, as well as the era of his presidency in the United States.\nThe Democratic Leadership Council, a pro-Democratic Party establishment, argues that Clintonism "stands for economic growth and opportunity; for fiscal responsibility; for work, not welfare; for preventing crime and punishing criminals; and for non-bureaucratic, empowering government" and further says that "these policies are key to the successes in the beginning of the 21st century."On the other hand, some critics of Clinton associate Clintonism with "coddling big money (except guns and tobacco), financial scandals, winning at any cost, flip-flopping and prevaricating".'

In [34]:
wikidata_de_ents_dataframe

,law,human1,human2,nat2,nat1,o2,o1,type,lawLabel
0,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':..."
1,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':..."
2,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':..."
3,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':..."
4,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':..."
...,...,...,...,...,...,...,...,...,...
5464,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':..."
5465,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':..."
5466,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'xml:lang': 'en', 'type': 'literal', 'value':..."
5467,"{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http://www.wikidata....","{'type': 'uri', 'value': 'http:/

In [1]:
import wikipedia
from bs4 import BeautifulSoup
import json
import unidecode
import re
import requests
from wikidata.client import Client
import spacy
import time

nlp = spacy.load('en_core_web_sm')

client = Client()

def generate_question(topic):
	topic = topic.replace("_", " ")
	try:
		context = wikipedia.summary(topic)
	except (wikipedia.exceptions.DisambiguationError,wikipedia.DisambiguationError) as e:
		s = e.options[0]
		context = wikipedia.summary(s)
	except:
		return None
	doc = nlp(context)
	# assert doc.has_annotation("SENT_START")
	first_sent = list(doc.sents)[0]
	for i,t in enumerate(first_sent):
		if t.text in ['is','was','are','were']:
			span = str(first_sent[i+1:])
			question = "Who discovered " + span
			return question
		elif t.text in ['describes','states','applies','establishes','gives']:
			span = str(first_sent[i:])
			question = "Who discovered a theorem that " + span
			return question
	if "used to" in str(first_sent):
		return "What equation can be used to {}".format(first_sent.split("used to")[1])

	return None

def get_wiki_summary(s):
	try:
		p = wikipedia.summary(s)
		return p
	except (wikipedia.exceptions.DisambiguationError,wikipedia.DisambiguationError) as e:
		s = e.options[0]
		p = wikipedia.summary(s)
		return p
	except:
		return s.replace(" ","_")

def get_wiki_name(wiki_guess):
	try:
		p = wikipedia.summary(wiki_guess)
		return wiki_guess
	except wikipedia.DisambiguationError as e:
		return e.options[0]
	except:
		return ''

def get_gender_nationality(wiki_name):
	try:
		a = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&titles={}&normalize=1&format=json".format(wiki_name))
		a = a.json()
		wiki_id = list(a['entities'].keys())[0]
		return get_wikidata(wiki_id)
	except:
		return {'gender': 'unknown', 'ethnicity': 'unknown','nationality': 'unknown'}

def find_names(law_name):
	law_name = law_name.replace("_", " ")
	names = law_name.split("-")
	name1 = names[0].split(" ")[-1].strip("() ,;")
	name2 = names[1].split(" ")[0].strip("() ,;")
	links = wikipedia.page(law_name).links
	summary = str(wikipedia.summary(law_name)).lower()
	potential_names = [i for i in links if ("mechanics" not in i.lower() and (name1.lower() in i.lower() or name2.lower() in i.lower()) and "–" not in i.lower())]
	return [i for i in potential_names if i.lower() in summary]

/fs/clip-quiz/flatearf/miniconda/miniconda3/envs/pyenv/lib/python3.9/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
import pandas as pd 
filename = 'wikidata_things_named_after_two_mixed_gender.json'
with open(filename,"r") as file:
    wikidata_de_ents = json.load(file)

In [9]:
wikidata_de_ents_dataframe = pd.DataFrame(wikidata_de_ents)


In [12]:
wikidata_de_ents_dataframe.groupby("lawLabel").agg(lambda x: list(x))

,law,human1,human2,nat1,nat2,o1,o2,type
lawLabel,,,,,,,,
135268 Haigneré,"[http://www.wikidata.org/entity/Q2641086, http...","[http://www.wikidata.org/entity/Q235580, http:...","[http://www.wikidata.org/entity/Q721397, http:...","[http://www.wikidata.org/entity/Q142, http://w...","[http://www.wikidata.org/entity/Q142, http://w...","[http://www.wikidata.org/entity/Q11631, http:/...","[http://www.wikidata.org/entity/Q11631, http:/...","[http://www.wikidata.org/entity/Q3863, http://..."
2325 Chernykh,"[http://www.wikidata.org/entity/Q516852, http:...","[http://www.wikidata.org/entity/Q234878, http:...","[http://www.wikidata.org/entity/Q318611, http:...","[http://www.wikidata.org/entity/Q159, http://w...","[http://www.wikidata.org/entity/Q212, http://w...","[http://www.wikidata.org/entity/Q11063, http:/...","[http://www.wikidata.org/entity/Q11063, http:/...","[http://www.wikidata.org/entity/Q3863, http://..."
3195 Fedchenko,"[http://www.wikidata.org/entity/Q666223, http:...","[http://www.wikidata.org/entity/Q3350415, http...","[http://www.wikidata.org/entity/Q258817, http:...","[http://www.wikidata.org/entity/Q34266, http:/...","[http://www.wikidata.org/entity/Q34266, http:/...","[http://www.wikidata.org/entity/Q483501, http:...","[http://www.wikidata.org/entity/Q18805, http:/...","[http://www.wikidata.org/entity/Q3863, http://..."
3484 Neugebauer,"[http://www.wikidata.org/entity/Q657366, http:...","[http://www.wikidata.org/entity/Q4316384, http...","[http://www.wikidata.org/entity/Q111996, http:...","[http://www.wikidata.org/entity/Q30, http://ww...","[http://www.wikidata.org/entity/Q30, http://ww...","[http://www.wikidata.org/entity/Q169470, http:...","[http://www.wikidata.org/entity/Q16063546, htt...","[http://www.wikidata.org/entity/Q3863, http://..."
4242 Brecher,"[http://www.wikidata.org/entity/Q153142, http:...","[http://www.wikidata.org/entity/Q108482630, ht...","[http://www.wikidata.org/entity/Q3195167, http...","[http://www.wikidata.org/entity/Q30, http://ww...","[http://www.wikidata.org/entity/Q30, http://ww...","[http://www.wikidata.org/entity/Q109123470, ht...","[http://www.wikidata.org/entity/Q11063, http:/...","[http://www.wikidata.org/entity/Q3863, http://..."
...,...,...,...,...,...,...,...,...
rue Robert-et-Sonia-Delaunay,"[http://www.wikidata.org/entity/Q3449750, http...","[http://www.wikidata.org/entity/Q232972, http:...","[http://www.wikidata.org/entity/Q33978, http:/...","[http://www.wikidata.org/entity/Q159, http://w...","[http://www.wikidata.org/entity/Q142, http://w...","[http://www.wikidata.org/entity/Q1028181, http...","[http://www.wikidata.org/entity/Q483501, http:...","[http://www.wikidata.org/entity/Q79007, http:/..."
rue de Galliera,"[http://www.wikidata.org/entity/Q3450741, http...","[http://www.wikidata.org/entity/Q287127, http:...","[http://www.wikidata.org/entity/Q3417354, http...","[http://www.wikidata.org/entity/Q172579, http:...","[http://www.wikidata.org/entity/Q172579, http:...","[http://www.wikidata.org/entity/Q3068305, http...","[http://www.wikidata.org/entity/Q43845, http:/...","[http://www.wikidata.org/entity/Q79007, http:/..."
rue des Docteurs-Déjérine,"[http://www.wikidata.org/entity/Q3451793, http...","[http://www.wikidata.org/entity/Q2870950, http...","[http://www.wikidata.org/entity/Q612744, http:...","[http://www.wikidata.org/entity/Q142, http://w...","[http://www.wikidata.org/entity/Q142, http://w...","[http://www.wikidata.org/entity/Q39631, http:/...","[http://www.wikidata.org/entity/Q39631, http:/...","[http://www.wikidata.org/entity/Q79007, http:/..."
